In [ ]:
from MotionGPT import *

In [ ]:
loader = FBXDataLoader("FBXData")

In [ ]:
loader.LoadData()

In [ ]:
len(loader.raw_data)

In [ ]:
len(loader.raw_data[0])

In [ ]:
loader.PrepareTrainingData(12)

In [ ]:
len(loader.train_data)

In [ ]:
len(loader.test_data)

In [ ]:
loader.test_data[0]

In [ ]:
len(loader.train_data[180])

In [ ]:
from VRNN import *

In [ ]:
input_dim = len(loader.train_data[0][1])
h_dim = 64
z_dim = 10
n_layers = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  #device

In [ ]:
print(device)

In [ ]:
model = VRNN(input_dim, h_dim, z_dim, n_layers, device)
model.to(device)

In [ ]:
batch_size = 16
n_epochs = 100
clip = 10
learning_rate = 1e-3
print_every = 10

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(n_epochs):
    batch_idx = 0   
    for batch_data, pad_data in loader.next_batch(batch_size):
        batch_idx += 1

        batch_data = batch_data.to(device)
        pad_data = pad_data.to(device)

        optimizer.zero_grad()

        kld_loss, nll_loss, _, _ = model(batch_data, pad_data)
        loss = kld_loss + nll_loss
        loss.backward()

        #grad norm clipping, only in pytorch version >= 1.10
        torch.nn.utils.clip_grad_norm(model.parameters(), clip)
        optimizer.step()

        #printing
        if batch_idx % print_every == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t KLD Loss: {:.6f} \t NLL Loss: {:.6f}'.format(
                epoch, batch_idx * len(batch_data), len(loader.train_data),
                100. * batch_idx / len(loader.train_data),
                kld_loss.data / batch_size,
                nll_loss.data / batch_size))

In [ ]:
model.sample(20)

-------------------------------------

In [1]:
from MotionGPT import *

In [2]:
maker = FBXDataMaker()

In [14]:
maker.mc.UndoToBeginning(10000)

(UndoToBeginning)Undo steps: 1351


In [4]:
maker.GetModelNameSpace()
print(maker.namespace)

mixamorig


In [5]:
maker.LoadModel("runs/2020_08_17_11_23_28_nr.pth")

In [15]:
maker.SampleAnim(30)